# 인덱싱 말고 zip() 함수 이용해서 다시 짜보기!

In [1]:
import re
import random
from tqdm import tqdm

NUM_LINES = 0
NUM_SRC_TOKENS = 0
NUM_TGT_TOKENS = 0
NUM_TOKENS_EQUAL_IN_LINE = 0
NUM_TOKENS_INEQUAL_IN_LINE = 0

ls = []
freq = dict()     # SRC_TO_TGT (EN기준 사전)
freq_ts = dict()  # TGT_TO_SRC (KO기준 사전)

fname = "06.enko.dict.person"
output_st = open(f'{fname}.SRC_TO_TGT', "w", encoding = 'utf8')
output_ts = open(f'{fname}.TGT_TO_SRC', "w", encoding = 'utf8')

with open(fname, encoding='utf8') as file:
    for In, line in tqdm(enumerate(file,1)):
        line = line.strip().split('\t')
        en, ko = line
        en_token = en.split(' ')
        ko_token = ko.split(' ')
        
        # NUM_LINES
        NUM_LINES += 1  # 총 48391 라인
        
        # NUM_SRC_TOKENS
        NUM_SRC_TOKENS += len(en_token)  # 총 98255 src 토큰
        
        # NUM_TGT_TOKENS
        NUM_TGT_TOKENS += len(ko_token)  # 총 91921 tgt 토큰
        
        # NUM_TOKENS_EQUAL_IN_LINE & NUM_TOKENS_INEQUAL_IN_LINE
        if len(ko_token) == len(en_token):
            NUM_TOKENS_EQUAL_IN_LINE += 1  # 총 42058 토큰 일치 라인
        else:
            # print(en, ko, sep='\t') # 어떤게 토큰수 불일치인지 확인 --> 모두 한국이름
            NUM_TOKENS_INEQUAL_IN_LINE += 1  # 총 6333 토큰 불일치 라인
        
        ls.append((In, en, ko))
        
        
        # SRC_TO_TGT 사전 value로 모두 리스트로 담아놓기
        if len(ko_token) == len(en_token):
            for w in en_token:
                if re.search('^[0-9]+$', w): continue
                if w not in freq:
                    freq[w] = [ko_token[en_token.index(w)]]
                else:
                    freq[w].extend([ko_token[en_token.index(w)]]) 
                    # extend([]) 안에 괄호 안써주면 따로따로 떼어져 들어감
        else:
            if en_token[0] not in freq:
                freq[en_token[0]] = [ko[0]]
            else:
                freq[en_token[0]].extend([ko[0]])
            if en_token[1] not in freq:
                freq[en_token[1]] = [ko[1:]]
            else:
                freq[en_token[1]].extend([ko[1:]])
        
        
        # TGT_TO_SRC 사전 value로 모두 리스트로 담아놓기
        if len(ko_token) == len(en_token):
            for w in ko_token:
                if re.search('^[0-9]+$', w): continue
                if w not in freq_ts:
                    freq_ts[w] = [en_token[ko_token.index(w)]]
                else:
                    freq_ts[w].extend([en_token[ko_token.index(w)]]) 
                    # extend([]) 안에 괄호 안써주면 따로따로 떼어져 들어감
        else:
            if ko[0] not in freq_ts:
                freq_ts[ko[0]] = [en_token[0]]
            else:
                freq_ts[ko[0]].extend([en_token[0]])
            if ko[1:] not in freq_ts:
                freq_ts[ko[1:]] = en_token[1:]
            else:
                freq_ts[ko[1:]].extend(en_token[1:])
        
        #print(In, line, sep='\t')

# print(freq)
# for x,y in sorted(freq.items(), key = lambda x:-x[1]):
#     print(x,y,sep='\t')

# SRC_TO_TGT 딕셔너리 속 딕셔너리 만들기 & 빈도 내림차순 정렬
for e_n in freq:
    Freq = {}
    for k_n in freq[e_n]:
        if k_n not in Freq:
            Freq[k_n] = 0
        Freq[k_n] += 1
    freq[e_n] = dict(sorted(Freq.items(), key = lambda x: (-x[1], x[0])))
    
freq = dict(sorted(freq.items(), key = lambda x: -sum(x[1].values())))
# # len(freq) # 42400개 키 (영어 이름)

# TGT_TO_SRC 딕셔너리 속 딕셔너리 만들기 & 빈도 내림차순 정렬
for k_n in freq_ts:
    Freq = {}
    for e_n in freq_ts[k_n]:
        if e_n not in Freq:
            Freq[e_n] = 0
        Freq[e_n] += 1
    freq_ts[k_n] = dict(sorted(Freq.items(), key = lambda x: (-x[1], x[0])))
    
freq_ts = dict(sorted(freq_ts.items(), key = lambda x: -sum(x[1].values())))
# len(freq_ts) # 40700개 키 (한국어 이름)


# MULTIPLE_TRANSLATIONS_IN_SRC_TO_TGT
for i in tqdm(freq):  # 약 6분 4초 소요
    print("MULTIPLE_TRANSLATIONS_IN_SRC_TO_TGT", i, file = output_st)
    for j in freq[i]:
        ex = random.sample(tuple((x,y,z) for x,y,z in ls if j in z), k=1)
        #rand_n = set(random.sample(range(0, len(ex)), k = 1))
        #and_n = random.randint(0, len(ex)-1)
        print("MULTIPLE_TRANSLATIONS_IN_SRC_TO_TGT", 
              tuple((j, freq[i][j], f'{freq[i][j]/sum(freq[i].values()):.2%}', *ex)), file = output_st)
    print(file = output_st)
# 약 131,962 라인

# MULTIPLE_TRANSLATIONS_IN_TGT_TO_SRC
for i in tqdm(freq_ts):  # 약 5분 21초 소요
    print("MULTIPLE_TRANSLATIONS_IN_TGT_TO_SRC", i, file = output_ts)
    for j in freq_ts[i]:
        ex = random.sample(tuple((x,y,z) for x,y,z in ls if j in y), k=1)
        #rand_n = set(random.sample(range(0, len(ex)), k = 1))
        #rand_n = random.randint(0, len(ex)-1)
        print("MULTIPLE_TRANSLATIONS_IN_TGT_TO_SRC", 
              tuple((j, freq_ts[i][j], f'{freq_ts[i][j]/sum(freq_ts[i].values()):.2%}', *ex)), file = output_ts)
    print(file = output_ts)
# 약 128,575 라인 (사전 끝 5개 키 잘리는 현상)


# with open("06.enko.dict.person", encoding='utf8') as file:
#     for In, line in enumerate(file, 1):
#         line = line.strip().split('\t')
#         en, ko = line
#         for i in freq:
#             for j in freq[i]:
#                 if j in ko:
#                     print(i, j, freq[i][j], f'{freq[i][j]/sum(freq[i].values()):.2%}', (In, en, ko))

48391it [00:00, 66496.60it/s]
100%|██████████| 40700/40700 [05:21<00:00, 126.67it/s]


In [43]:
freq

{'Kim': {'김': 1207, '킴': 41, 'ko': 1, '상기': 1, '은성': 1, '재익': 1},
 'Lee': {'이': 858, '리': 110},
 'Park': {'박': 513, '파크': 2},
 'John': {'존': 456,
  '요한': 4,
  '욘': 4,
  '장': 3,
  '요안니스': 2,
  '얀': 1,
  '요한네스': 1,
  '존.': 1,
  '주앙': 1,
  '혼': 1,
  '후안': 1},
 'de': {'드': 164, '데': 106, '더': 24, '지': 13, '디': 4, '덕': 2, '다': 1, '제': 1},
 'Jung': {'정': 289, '융': 1, '중': 1},
 'Choi': {'최': 273, '초이': 1},
 'David': {'데이비드': 231, '데이빗': 16, '다비드': 14, '다비트': 7},
 'Michael': {'마이클': 210,
  '미하엘': 23,
  '미카엘': 7,
  '미하일': 5,
  '마이크': 1,
  '미이클': 1,
  '미차엘': 1,
  '미하얼': 1,
  '미하일로': 1},
 'Robert': {'로버트': 215,
  '로베르': 14,
  '로베르트': 14,
  '로버르트': 2,
  '로베르토': 1,
  '롭': 1,
  '호베르트': 1},
 'Cho': {'조': 225, '장': 1, '초': 1},
 'Paul': {'폴': 200, '파울': 16, '바오로': 2},
 'James': {'제임스': 203, '하이메': 1},
 'William': {'윌리엄': 181, '빌헬름': 4, '윌리암': 3, '기욤': 1, '빌': 1, '윌리앙': 1},
 'Kang': {'강': 186, '캉': 4, '항': 1},
 'Peter': {'피터': 149,
  '페터': 19,
  '페테르': 10,
  '페터르': 5,
  '베드로': 2,
  '표트르': 2,
  '피에르': 2,

In [48]:
freq_ts

{'김': {'Kim': 1207,
  'Chae-ik': 1,
  'Dae': 1,
  'Eunseong': 1,
  'Jin': 1,
  'Kim-Jho': 1,
  'Sanggi': 1},
 '이': {'Lee': 858,
  'Yi': 35,
  'Li': 25,
  'Y': 6,
  'I': 5,
  'y': 2,
  'E': 1,
  'Rhee': 1,
  'Tự': 1},
 '박': {'Park': 513, 'Pak': 5, 'Bak': 2, 'Pahk': 1},
 '존': {'John': 456, 'Jon': 30, 'Joan': 1, 'Jón': 1, 'Zoon': 1, 'Zun': 1},
 '조': {'Cho': 225,
  'Joe': 62,
  'Cao': 17,
  'Jo': 15,
  'Zoe': 8,
  'Zhao': 4,
  'Chao': 3,
  'Zu': 2,
  'Jho': 1,
  'Jō': 1,
  'Zuo': 1},
 '정': {'Jung': 289,
  'Jeong': 11,
  'Zheng': 8,
  'Ding': 6,
  'Jing': 6,
  'Cheng': 5,
  'Chung': 4,
  'Trinh': 2,
  'Mi-jung': 1,
  'Ting': 1,
  'Trịnh': 1},
 '장': {'Jang': 144,
  'Jean': 98,
  'Zhang': 52,
  'Jiang': 11,
  'Chang': 10,
  'John': 3,
  'Chiang': 1,
  'Cho': 1,
  'Jan': 1,
  'Ki-yong': 1,
  'Trang': 1},
 '최': {'Choi': 273, 'Choe': 1, 'Cui': 1},
 '리': {'Lee': 110,
  'Ri': 63,
  'Li': 45,
  'Leigh': 8,
  'Ly': 5,
  'Le': 2,
  'Lý': 2,
  'Lei': 1,
  'Lie': 1,
  'Ry': 1,
  'Rì': 1},
 '데이비드': {'Da

In [5]:
NUM_LINES

48391

In [6]:
NUM_SRC_TOKENS

98255

In [7]:
NUM_TGT_TOKENS

91921

In [9]:
NUM_TOKENS_INEQUAL_IN_LINE

6333

In [10]:
NUM_TOKENS_EQUAL_IN_LINE

42058